# Содержание
1. [Описание проекта](#description),
2. [Открытие данных](#1),
3. [Предобработка данных](#2),
   * [Обработка пропусков](#3),
   * [Замена типа данных](#4),
   * [Обработка дубликатов](#5),
   * [Лемматизация](#6),
   * [Категоризация данных](#7),
4. [Ответы на вопросы](#8),
5. [Общий вывод](#9),


## Описание проекта  <a id='description'></a>

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию.  <a id='1'></a>

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()
display(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Информация о таблице получена методом df.info(). Поменять тип данных days_employed на целочисленный, для удобства подсчета стажа. df.head(10) выводит первые 10 строк таблицы. В total_income выводить 2 знака после запятой, так как это денежное выражение. Привести к нижнему регистру.

### Шаг 2. Предобработка данных <a id='2'></a>

### Обработка пропусков <a id='3'></a>
Подсчет суммы пропущенных строк методом isnull().sum()

In [2]:
print(df.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Пропущенные значения присутствуют в двух столбцах days_employed и total_sum, их количество совпадает. Проверим совпадают ли пропущенные значения.

In [3]:
print(df[df['days_employed'].isnull()].head(10))

    children  days_employed  dob_years education  education_id  \
12         0            NaN         65   среднее             1   
26         0            NaN         41   среднее             1   
29         0            NaN         63   среднее             1   
41         0            NaN         50   среднее             1   
55         0            NaN         54   среднее             1   
65         0            NaN         21   среднее             1   
67         0            NaN         52    высшее             0   
72         1            NaN         32    высшее             0   
82         2            NaN         50    высшее             0   
83         0            NaN         52   среднее             1   

            family_status  family_status_id gender  income_type  debt  \
12       гражданский брак                 1      M    пенсионер     0   
26        женат / замужем                 0      M  госслужащий     0   
29  Не женат / не замужем                 4      F    

Видно что пропущенные значения в столбцах совпадают. Проверим у каких групп заемщиков есть пропущенные значения и просуммируем их.

In [4]:
print(df[df['total_income'].isnull()].groupby('income_type').count())

                 children  days_employed  dob_years  education  education_id  \
income_type                                                                    
госслужащий           147              0        147        147           147   
компаньон             508              0        508        508           508   
пенсионер             413              0        413        413           413   
предприниматель         1              0          1          1             1   
сотрудник            1105              0       1105       1105          1105   

                 family_status  family_status_id  gender  debt  total_income  \
income_type                                                                    
госслужащий                147               147     147   147             0   
компаньон                  508               508     508   508             0   
пенсионер                  413               413     413   413             0   
предприниматель              1         

Пропущенные значения есть у госслужащих, компьонов, пенсионеров, предпринимателей, сотрудников. Заменим пропущенные значения медианным. Для подсчета среднего значения создадим датафрейм без пропущенных значений.

In [5]:
df_dropna = df.dropna().reset_index(drop = True)
print(df_dropna.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19351 entries, 0 to 19350
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null float64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null float64
purpose             19351 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 1.8+ MB
None


In [6]:
print(df_dropna.groupby('income_type')['total_income'].median())

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64


Сохраним каждое медианное значения в переменные.

In [7]:
grouped_df = df_dropna.groupby('income_type').agg({'total_income' : ['median']})
state_employee = grouped_df['total_income'].loc[:, 'median'][2]
companion = grouped_df['total_income'].loc[:, 'median'][3]
pensioner = grouped_df['total_income'].loc[:, 'median'][4]
businessman = grouped_df['total_income'].loc[:, 'median'][5]
employee = grouped_df['total_income'].loc[:, 'median'][6]
print(state_employee)
print(companion)
print(pensioner)
print(businessman)
print(employee)

150447.9352830068
172357.95096577113
118514.48641164352
499163.1449470857
142594.39684740017


Заменим пропущенные значения в столбце days_employed на 0 методом fillna() и заменим значения в total_income на среднее. Проверим пропущенные значения в таблице методом info().

In [8]:
df['days_employed'] = df['days_employed'].fillna(0)
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'госслужащий'), 'total_income'] = state_employee
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'компаньон'), 'total_income'] = companion
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'пенсионер'), 'total_income'] = pensioner
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'предприниматель'), 'total_income'] = businessman
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'сотрудник'), 'total_income'] = employee
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

Подсчет пропусков получен методом isnull().sum(). Пропущенные значения присутствовали в двух столбцах: days_employed и total_income. Принято решение заполнить пропущенные значения в days_employed на 0, в total_income на медианные.

### Замена типа данных <a id='4'></a>

В целом типы данных определены правильно. Приведем days_employed и total_income к целочисленому виду.

In [9]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
print(df.head())
print(df.info())

   children  days_employed  dob_years education  education_id  \
0         1          -8437         42    высшее             0   
1         1          -4024         36   среднее             1   
2         0          -5623         33   Среднее             1   
3         3          -4124         32   среднее             1   
4         0         340266         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M   сотрудник     0        145885   
3   женат / замужем                 0      M   сотрудник     0        267628   
4  гражданский брак                 1      F   пенсионер     0        158616   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жилья  


### Вывод

Тип данных float двух столбцов был заменен на тип данных int.

### Обработка дубликатов <a id='5'></a>

Проверим количество дубликатов.

In [10]:
print(df.duplicated().sum())

54


Приведем регистр записей к единому виду - нижнему регистру, методом str.lower().

In [11]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()
print(df.duplicated().sum())

71


Количество дубликатов 71, удалим данные повторы из таблицы и сбросим индексацию.

In [12]:
df = df.drop_duplicates().reset_index(drop = True)
print(df.duplicated().sum())

0


### Вывод

Дубликаты были удалены, индексация строк сброшена.

### Лемматизация <a id='6'></a>

Лемматизацию выполним для столбца purpose, чтобы определить какие слова использованы для описания получения кредита.

In [13]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
list = df['purpose'].tolist()

def listToString(list):
    str = ' '
    return (str.join(list))

text = listToString(list)
lemmas = m.lemmatize(text)
print(Counter(list))

Counter({'свадьба': 791, 'на проведение свадьбы': 768, 'сыграть свадьбу': 765, 'операции с недвижимостью': 675, 'покупка коммерческой недвижимости': 661, 'операции с жильем': 652, 'покупка жилья для сдачи': 651, 'операции с коммерческой недвижимостью': 650, 'покупка жилья': 646, 'жилье': 646, 'покупка жилья для семьи': 638, 'строительство собственной недвижимости': 635, 'недвижимость': 633, 'операции со своей недвижимостью': 627, 'строительство жилой недвижимости': 624, 'покупка недвижимости': 621, 'покупка своего жилья': 620, 'строительство недвижимости': 619, 'ремонт жилью': 607, 'покупка жилой недвижимости': 606, 'на покупку своего автомобиля': 505, 'заняться высшим образованием': 496, 'автомобиль': 494, 'сделка с подержанным автомобилем': 486, 'на покупку подержанного автомобиля': 478, 'автомобили': 478, 'свой автомобиль': 478, 'на покупку автомобиля': 471, 'приобретение автомобиля': 461, 'дополнительное образование': 460, 'сделка с автомобилем': 455, 'высшее образование': 452, 'об

### Вывод

Из подсчета встречаемости значений в списке видно, что цели записаны в различных вариациях. Эти данные можно будет использовать для категоризации данных таблицы по целям выдачи кредита.

### Категоризация данных <a id='7'></a>

In [14]:
purpose_list = []

for purpose in lemmas:
    if purpose == 'свадьба':
        purpose_list.append('свадьба')
    elif purpose == 'недвижимость' or purpose == 'жилье':
        purpose_list.append('недвижимость')
    elif purpose == 'автомобиль':
        purpose_list.append('автомобиль')
    elif purpose == 'образование':
        purpose_list.append('образование')

df['purpose_category'] = pd.Series(purpose_list)
print(df.head(10))

   children  days_employed  dob_years education  education_id  \
0         1          -8437         42    высшее             0   
1         1          -4024         36   среднее             1   
2         0          -5623         33   среднее             1   
3         3          -4124         32   среднее             1   
4         0         340266         53   среднее             1   
5         0           -926         27    высшее             0   
6         0          -2879         43    высшее             0   
7         0           -152         50   среднее             1   
8         2          -6929         35    высшее             0   
9         0          -2188         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

Разделим всех заемщиков на 2 категории: группа с детьми и без.

In [15]:
def children_group(children):
    if children == 0:
        try:
            return 'нет детей'
        except:
            return 'неверно заполненные данные о детях'
    if children >= 1:
        try:
            return 'есть дети'
        except:
            return 'неверно заполненные данные о детях'

df['children_group'] = df['children'].apply(children_group)
print(df.head(10))

   children  days_employed  dob_years education  education_id  \
0         1          -8437         42    высшее             0   
1         1          -4024         36   среднее             1   
2         0          -5623         33   среднее             1   
3         3          -4124         32   среднее             1   
4         0         340266         53   среднее             1   
5         0           -926         27    высшее             0   
6         0          -2879         43    высшее             0   
7         0           -152         50   среднее             1   
8         2          -6929         35    высшее             0   
9         0          -2188         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

Проверим необходимость категоризации данных по семейному положению.

In [16]:
print(df['family_status'].value_counts())

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


Все заемщики разделены на смысловые группы.
Проведем категоризацию по уровню дохода. Разделим столбец total_income на 3 категории: низкий доход, средний доход, высокий доход.

In [17]:
def income_level(income):
    if income <= 30000:
        return 'низкий доход'
    if 30000 < income <= 80000:
        return 'средний доход'
    if income > 80000:
        return 'высокий доход'

df['income_level'] = df['total_income'].apply(income_level)
print(df['income_level'].value_counts())

высокий доход    19178
средний доход     2254
низкий доход        22
Name: income_level, dtype: int64


Способ распределения по доходу не лучший, большинство заемщиков на уровне высокого дохода. Чтобы распределить группы посчитаем среднее значение.

In [18]:
print(df['total_income'].mean())

165319.57229421087


In [19]:
print(df[df['total_income'] < 160000]['total_income'].median())
print(df[df['total_income'] > 160000]['total_income'].median())
print(df[df['total_income'] < 100000]['total_income'].median())
print(df[df['total_income'] > 200000]['total_income'].median())

115441.0
212990.5
79505.0
257994.0


In [20]:
def income_level(income):
    if income <= 100000:
        return 'низкий доход'
    if 100000 < income <= 250000:
        return 'средний доход'
    if income > 250000:
        return 'высокий доход'

df['income_level'] = df['total_income'].apply(income_level)
print(df['income_level'].value_counts())

средний доход    14178
низкий доход      4463
высокий доход     2813
Name: income_level, dtype: int64


### Вывод

Таблицы прошли категоризацию по нескольким признакам: наличие или отсутствие детей, семейное положение, уровень дохода, цель получения кредита.

### Шаг 3. Ответьте на вопросы <a id='8'></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Посчитаем количество неоплаченных кредитов в срок, сгруппировав заемщиков по наличию детей.

In [21]:
print(df.groupby('children_group')['debt'].sum())

children_group
есть дети     677
нет детей    1063
Name: debt, dtype: int64


Посчитаем общее количесто заемщиков у которых нет детей и у которых есть дети.

In [22]:
df['children_group'].value_counts()

нет детей    14091
есть дети     7316
Name: children_group, dtype: int64

Подготовим списки для подсчета процентов неоплаченных кредитов в срок у двух групп заемщиков. Создаlим датафрейм, в который добавим столбец для расчета процента неоплаченных кредитов в срок.

In [23]:
children_group = df['children_group'].value_counts().tolist()
children_debt = df.groupby('children_group')['debt'].sum().sort_values(ascending=False).tolist()
final_list = [[children_group[0], children_debt[0]], [children_group[1], children_debt[1]]]
#print(final_list)
columns_children = ['group', 'debt']
children_debt = pd.DataFrame(data = final_list, columns = columns_children)
#print(children_debt)
children_debt['percent_debt'] = children_debt['debt'] / children_debt['group'] * 100
print(children_debt)

   group  debt  percent_debt
0  14091  1063      7.543822
1   7316   677      9.253691


In [24]:
children_d = df.groupby('children_group')['debt'].agg(group = 'count',  debt = 'sum')
children_d['percent'] = children_d['debt'] / children_d['group'] * 100
print(children_d)

                group  debt   percent
children_group                       
есть дети        7316   677  9.253691
нет детей       14091  1063  7.543822


### Вывод

У заемщиков у которых дети есть, результат хуже - 9.25%, чем у тех у кого нет детей - 7,54%. Зависимость есть

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Посчитаем количество неоплаченных кредитов в срок, сгруппировав заемщиков по семейному статусу и отсортируем данные для наглядности.

In [25]:
print(df.groupby('family_status')['debt'].sum().sort_values(ascending = False))

family_status
женат / замужем          931
гражданский брак         388
не женат / не замужем    274
в разводе                 85
вдовец / вдова            63
Name: debt, dtype: int64


Посчитаем общее количесто заемщиков из данных категорий.

In [26]:
df['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Подготовим списки для подсчета процентов неоплаченных кредитов в срок у двух групп заемщиков. Создаlим датафрейм, в который добавим столбец для расчета процента неоплаченных кредитов в срок.

In [27]:
family_status = df['family_status'].value_counts().tolist()
family_debt = df.groupby('family_status')['debt'].sum().sort_values(ascending=False).tolist()

second_final = []
for i in range(len(family_status)):
    second_final.append([family_status[i], family_debt[i], family_debt[i] / family_status[i] *100])
#print(second_final)

columns_family = ['status_count', 'status_debt', 'percent']
family_debt = pd.DataFrame(data = second_final, columns = columns_family)
print(family_debt)

   status_count  status_debt   percent
0         12339          931  7.545182
1          4151          388  9.347145
2          2810          274  9.750890
3          1195           85  7.112971
4           959           63  6.569343


In [28]:
family_d = df.groupby('family_status')['debt'].agg(status_count = 'count', status_debt = 'sum')
family_d['percent'] = family_d['status_debt'] / family_d['status_count'] * 100
print(family_d)

                       status_count  status_debt   percent
family_status                                             
в разводе                      1195           85  7.112971
вдовец / вдова                  959           63  6.569343
гражданский брак               4151          388  9.347145
женат / замужем               12339          931  7.545182
не женат / не замужем          2810          274  9.750890


### Вывод

Наибольший процент неоплаченных кредитов отмечается у двух групп: у тех, кто не женат/не замужем (9,75%) и у тех, кто состоит в гражданском браке (9,35%). Зависимость есть.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [29]:
print(df.groupby('income_level')['debt'].sum().sort_values(ascending=False))
df['income_level'].value_counts()

income_level
средний доход    1193
низкий доход      354
высокий доход     194
Name: debt, dtype: int64


средний доход    14178
низкий доход      4463
высокий доход     2813
Name: income_level, dtype: int64

In [30]:
income_level = df['income_level'].value_counts().tolist()
income_debt = df.groupby('income_level')['debt'].sum().sort_values(ascending=False).tolist()

third_final = []
for i in range(len(income_level)):
    third_final.append([income_level[i], income_debt[i], income_debt[i] / income_level[i] *100])
    
columns_income = ['income_level_count', 'debt_count', 'percent']
income_debt = pd.DataFrame(data = third_final, columns = columns_income)
print(income_debt)

   income_level_count  debt_count   percent
0               14178        1193  8.414445
1                4463         354  7.931884
2                2813         194  6.896552


In [31]:
incom_d = df.groupby('income_level')['debt'].agg(income_level_count = 'count', debt_count = 'sum')
incom_d['percent'] = incom_d['debt_count'] / incom_d['income_level_count'] * 100
print(incom_d)

               income_level_count  debt_count   percent
income_level                                           
высокий доход                2813         194  6.896552
низкий доход                 4463         354  7.931884
средний доход               14178        1193  8.414445


### Вывод

Зависимость есть. Наибольшая для непогашенных в срок кредитов у заемщиков со средним доходом - 8.41%, наименьшая у заемщиков с высоким доходом - 6.9%.

- Как разные цели кредита влияют на его возврат в срок?

In [32]:
print(df.groupby('purpose_category')['debt'].sum().sort_values(ascending=False))
df['purpose_category'].value_counts()

purpose_category
недвижимость    782
автомобиль      403
образование     370
свадьба         186
Name: debt, dtype: int64


недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

In [33]:
purpose_list = df['purpose_category'].value_counts().tolist()
purpose_debt = df.groupby('purpose_category')['debt'].sum().sort_values(ascending=False).tolist()
category_list = df['purpose_category'].drop_duplicates().reset_index(drop = True).to_dict()

fourth_final = []
for i in range(len(purpose_list)):
    fourth_final.append([purpose_list[i], purpose_debt[i], purpose_debt[i] / purpose_list[i] *100])

    
columns_purpose = ['purpose_count', 'debt_count', 'percent']
purpose_debt = pd.DataFrame(data = fourth_final, columns = columns_purpose)
#purpose_debt.index = ['недвижимость', 'автомобиль', 'образование', 'свадьба']
purpose_debt = purpose_debt.rename(index = category_list)
print(purpose_debt)

              purpose_count  debt_count   percent
недвижимость          10811         782  7.233373
автомобиль             4306         403  9.359034
образование            4013         370  9.220035
свадьба                2324         186  8.003442


In [34]:
purpose_d = df.groupby('purpose_category')['debt'].agg(purpose_count = 'count', debt_count = 'sum')
purpose_d['percent'] = purpose_d['debt_count'] / purpose_d['purpose_count'] * 100
print(purpose_d)

                  purpose_count  debt_count   percent
purpose_category                                     
автомобиль                 4306         403  9.359034
недвижимость              10811         782  7.233373
образование                4013         370  9.220035
свадьба                    2324         186  8.003442


In [35]:
tot = df.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = ({'sum' , 'count'}))
#print(tot.head(10))
tot['percent'] = tot['sum'] / tot['count'] * 100
tot.rename(columns = {'count': 'purpose_count', 'sum': 'debt_count'}, inplace=True)
display(tot)

,purpose_count,debt_count,percent
purpose_category,,,
автомобиль,4306,403,9.359034
недвижимость,10811,782,7.233373
образование,4013,370,9.220035
свадьба,2324,186,8.003442


### Вывод

Наибольшая доля непогашенных кредитов приходится на категорию автомобиля - 9.36% и на образование - 9.22%. Наименьший на жилье - 7.23%.

In [36]:
# код ревьювера
# таблица зависимости медианного заработка от образования и вида деятельности
df.pivot_table(index = 'education', columns ='income_type', values = 'total_income', aggfunc = 'median').fillna(0)

income_type,безработный,в декрете,госслужащий,компаньон,пенсионер,предприниматель,сотрудник,студент
education,,,,,,,,
высшее,202722.0,0.0,158384.5,191266.5,137165.0,499163.0,155135.0,98201.0
начальное,0.0,0.0,150447.0,150100.5,107398.0,0.0,131629.0,0.0
неоконченное высшее,0.0,0.0,156266.5,172357.0,118514.0,0.0,144499.0,0.0
среднее,59956.0,53829.0,144350.5,167630.0,118514.0,0.0,142594.0,0.0
ученая степень,0.0,0.0,111392.0,0.0,177088.5,0.0,198570.0,0.0


### Шаг 4. Общий вывод <a id='9'></a>

Произведена обработка данных для дальнейшего анализа: убраны пропуски, удалены дубликаты, изменен тип данных с вещественного на целочисленный, лемматизация и категоризация данных.